# Fine Tune Legal LLM
We are going to fine-tune a fine-tuned model for legal texts, using [unsloth](https://github.com/unslothai/unsloth).

1. Download unsloth and required libraries
2. Load the specialized [model](https://huggingface.co/StevenChen16/llama3-8b-Lawyer)
3. Import the [structured UAE laws dataset](https://huggingface.co/datasets/obadabaq/structured-uae-laws)
4. Train the model using [unsloth notebook](https://colab.research.google.com/drive/135ced7oHytdxu3N2DNe1Z0kqjyYIkDXp?usp=sharing)
5. Save the model to [huggingface](https://huggingface.co/obadabaq/ai_lawyer) 

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "StevenChen16/llama3-8b-Lawyer",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00009.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

StevenChen16/llama3-8b-Lawyer does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [2]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# import pandas as pd
# from datasets import Dataset
# df = pd.read_csv('structured-uae-laws.csv')
# dataset = Dataset.from_pandas(df)
# dataset = dataset.map(formatting_prompts_func, batched = True,)

from datasets import load_dataset
dataset = load_dataset("obadabaq/structured-uae-laws", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

ModuleNotFoundError: No module named 'datasets'

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/9446 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,446 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.760700
2,2.322400
3,2.294100
4,2.358500
5,2.068600
6,1.839400
7,1.830700
8,1.636200
9,1.610300
10,1.650100


In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What are the steps to start a free-zone company in the UAE?",
        "",
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What are the steps to start a free-zone company in the UAE?

### Input:


### Response:
To start a free-zone company in the UAE, you must follow these steps:
1. Choose a free zone that suits your business needs and activities.
2. Obtain a license from the relevant free zone authority.
3. Set up a company in the free zone, which may involve registering a new entity or transferring an existing one.
4. Obtain an entry visa for yourself and any other shareholders or employees who will be working in the free zone.
5. Open a corporate bank account in the UAE.
6. Obtain a residence visa for yourself and any other shareholders or employees who will be working in the free zone.
7. Obtain a trade


In [12]:
model.save_pretrained("ai_lawyer")
tokenizer.save_pretrained("ai_lawyer")
model.push_to_hub("obadabaq/ai_lawyer", token = "") # add the huggingface token here
tokenizer.push_to_hub("obadabaq/ai_lawyer", token = "") # add the huggingface token here

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/obadabaq/ai_lawyer
